In [1]:
import pandas as pd
import os

# Constants
QUESTION_TYPE = 1
ANSWER_TYPE = 2

file = os.path.join('..', 'data', 'pandas-code-dataset.csv')

dataset = pd.read_csv(file, encoding='ISO-8859-1', error_bad_lines=False, sep=";")
print('Amount of entries in the dataset = %d' % len(dataset))
dataset

Amount of entries in the dataset = 3205


,Unnamed: 0,AnswerCount,CommentCount,Id,ParentId,PostTypeId,Score,Tags,Title,Body,ViewCount,Code,CodeList,AllCode
0,0,2.0,7,7776679,NaN,1,25,<python><pandas>,append two data frame with pandas,<p>When I try to merge two dataframes by rows ...,25479.0,['bigdata = data1.append(data2)\nException: In...,"['bigdata = data1.append(data2)\n', 'Exception...","['bigdata = data1.append(data2)\n', 'Exception..."
1,1,7.0,3,7837722,NaN,1,187,<python><performance><for-loop><pandas>,What is the most efficient way to loop through...,<p>I want to perform my own complex operations...,195364.0,"[""Date,Open,High,Low,Close,Volume,Adj Close\n2...","['Date,Open,High,Low,Close,Volume,Adj Close\n2...","['Date,Open,High,Low,Close,Volume,Adj Close\n2..."
2,2,2.0,0,8916302,NaN,1,26,<python><csv><numpy><tab-delimited><pandas>,selecting across multiple columns with python ...,<p>I have a dataframe <code>df</code> in panda...,19805.0,"['df_greater_than10 = df[df[""colA""] > 10]\n']","['df_greater_than10 = df[df[""colA""] > 10]\n']","['df', 'pandas.read_table', 'colA', 'df_greate..."
3,3,3.0,16,8991709,NaN,1,134,<python><r><join><data.table><pandas>,Why are pandas merges in python faster than da...,"<p>I recently came across the <a href=""http://...",16173.0,[''],[],"['pandas', 'data.table', 'data.table', 'merge(..."
4,4,8.0,5,9555635,NaN,1,19,<python><numpy><scipy><enthought><pandas>,Open source Enthought Python alternative,<p>I used Enthought's python distribution as a...,6585.0,[''],[],"['numpy', 'scipy', 'sci-kits']"
5,5,2.0,2,9588331,NaN,1,21,<python><pandas>,Simple cross-tabulation in pandas,"<p>I stumbled across <a href=""http://pandas.py...",9493.0,"['AB,100.00\nAB,200.00\nAC,150.00\nAD,500.00\n...","['AB,100.00\nAB,200.00\nAC,150.00\nAD,500.00\n...","['pivot/crosstab/indexing', 'Panel', 'DataFram..."
6,6,3.0,0,9652832,NaN,1,38,<python><pandas><tsv>,How to I load a tsv file into a Pandas DataFrame?,<p>I'm new to python and pandas. I'm trying t...,32040.0,['>>> df1 = DataFrame(csv.reader(open(\'c:/~/t...,['>>> df1 = DataFrame(csv.reader(open(\'c:/~/t...,"['tsv', 'DataFrame', '>>> df1 = DataFrame(csv...."
7,7,6.0,0,9758450,NaN,1,34,<python><pandas>,Pandas convert dataframe to array of tuples,<p>I have manipulated some data using pandas a...,36433.0,['In [182]: data_set\nOut[182]: \n index data...,['In [182]: data_set\nOut[182]: \n index data...,['In [182]: data_set\nOut[182]: \n index data...
8,8,2.0,0,9762935,NaN,1,11,<python><pandas>,Add indexed column to DataFrame with pandas,"<p>I'm a beginning pandas user, and after stud...",8189.0,['df\n\n A ...,['df\n\n A ...,['df\n\n A ...
9,9,2.0,0,9787853,NaN,1,17,<python><pandas>,join or merge with overwrite in pandas,<p>I want to perform a join/merge/append opera...,6054.0,[''],[],"['df1', 'df2', 'df2', 'df2', 'df1', 'df1', 'df2']"


In [2]:
import re

text = """ Example of code to be filtered (only in In[11])
In [11]: df
df.append()
Out[11]: 
A B C D 
2000-01-03 -0.59885 -0.18141 -0.68828 -0.77572
2000-01-04 0.83935 0.15993 0.95911 -1.12959

In [12]: df[(df.values > 1.5).any(1)]
Out[12]: 
A B C D 
2000-01-05 2.8021 -0.1086 -1.62114 -0.2017
2000-01-06 0.7167 -0.2671 1.36029 1.7425
"""

def preprocessTerminalLikeCode(code):
    terminalRegex='In \[.+?\]:(.*?)Out\[.+?\]'
    sequence = re.compile(terminalRegex, re.DOTALL)
    terminalCode = sequence.findall(code)
    return [i.lstrip() for i in terminalCode]

In [3]:
import ast

text = """ Example of code to be filtered (only in In[11])
df
df.append()

A B C D 
2000-01-03 -0.59885 -0.18141 -0.68828 -0.77572
2000-01-04 0.83935 0.15993 0.95911 -1.12959

df[(df.values > 1.5).any(1)]
 
A B C D 
2000-01-05 2.8021 -0.1086 -1.62114 -0.2017
2000-01-06 0.7167 -0.2671 1.36029 1.7425
"""

def preprocessOutputCode(code):
    lines = code.split('\n')
    preprocessed = []
    for l in lines:
        try:
            ast.parse(l)
            preprocessed.append(l)
        except:
            # Ignore
            pass
    return '\n'.join(preprocessed)
            
# preprocessOutputCode(text)


In [4]:
import ast

def preprocessCode(code):
    # First Method for pre-processing
    terminalLikeCode = preprocessTerminalLikeCode(code)
    if terminalLikeCode:
        code = ''.join(terminalLikeCode)
        
    code = preprocessOutputCode(code)
    return code
    
def preprocessPost(post, col):
    strcodes = post[col]
    codes = ast.literal_eval(strcodes)
    processed = []
    for code in codes:
        processed.append(preprocessCode(code))
    post['ProcessedCode'] = processed
    return post


processed_dataset = dataset.apply(preprocessPost, axis=1, args=('CodeList',))


processed_dataset.to_csv(os.path.join('..', 'data', 'pandas-processedcode-dataset.csv'), encoding='ISO-8859-1', sep=';', doublequote=True)
